In [1]:
import cv2
import numpy as np

In [2]:
capture = cv2.VideoCapture(0)

In [3]:
ret, prev_frame = capture.read()
# previous frame to match
prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)    

while True:
    ret, curr_frame = capture.read()
    curr_frame_rgb_copy = np.copy(curr_frame)
    curr_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)    
    
    # getting the difference to find motion
    difference = cv2.absdiff(curr_frame,prev_frame)
    #thresholding for finding contours
    _,difference = cv2.threshold(difference,50,255,cv2.THRESH_BINARY)
    #getting all the contours
    contours,_ = cv2.findContours(difference, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #with all the contours we will make the largest rectangle bounding these contours
    #basically all the white portion of the "difference" should be inside it
    height, width = difference.shape
    min_x, min_y = width, height
    max_x = max_y = 0
    
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        #getting all the boudaries of the contours & finding the min & max for final rectangle
        min_x, max_x = min(x, min_x), max(x+w, max_x)
        min_y, max_y = min(y, min_y), max(y+h, max_y)
    
    #drawing the final rectangle
    if max_x - min_x > 0 and max_y - min_y > 0:
        cv2.rectangle(curr_frame_rgb_copy, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)
        
    cv2.imshow('Motion Detection', curr_frame_rgb_copy)
    prev_frame = np.copy(curr_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
capture.release()
cv2.destroyAllWindows()